In [ ]:
from utils_detectron2 import visualize_annotations, visualize_predictions, evaluate_predictions, move_train_artifacts_to_drive, LossEvalHook
from utils_environment import load_classes_from_drive

class_name_to_id_mapping, class_names = load_classes_from_drive()

# CHDIR TO DETR REPO
import os
os.chdir("detr")

COLAB = False

from detectron2.config import get_cfg
from detectron2.engine import BestCheckpointer
from detectron2.data import DatasetMapper, build_detection_test_loader
from detectron2.data.datasets import register_coco_instances
from detectron2.checkpoint import DetectionCheckpointer
from d2.detr import add_detr_config
from d2.train_net import Trainer

%matplotlib inline

In [ ]:
def create_trainer_validation_best_checkpoint(cfg):
    class MyTrainer(Trainer): 
        def build_hooks(self):
            hooks = super().build_hooks()

            # Evaluate on VAL Set 
            hooks.insert(-1,LossEvalHook(
                cfg.TEST.EVAL_PERIOD,
                self.model,
                build_detection_test_loader(
                    self.cfg,
                    self.cfg.DATASETS.TEST[0],
                    DatasetMapper(self.cfg,True)
                )
            ))

            # Save bestcheckpoint on BBOX/AP50 Metric on VAL SET
            hooks.insert(-1, BestCheckpointer(cfg.TEST.EVAL_PERIOD,
                DetectionCheckpointer(self.model, cfg.OUTPUT_DIR),
                "bbox/AP50",
                "max",
                ))
            return hooks

    return MyTrainer(cfg)
        
def get_default_cfg(class_names):
    cfg = get_cfg()
    add_detr_config(cfg)
    cfg.merge_from_file("d2/configs/detr_256_6_6_torchvision.yaml")
    cfg.DATASETS.TRAIN = ("cghd_train",)
    cfg.DATASETS.TEST = ("cghd_val",)
    cfg.DATALOADER.NUM_WORKERS = 0

    cfg.MODEL.MASK_ON = False
    cfg.MODEL.DETR.NUM_CLASSES = 45
    
    cfg.SOLVER.STEPS = []        # do not decay learning rate
    cfg.TEST.EVAL_PERIOD = 20
    
    # NO AUGMENTATION fixed imgsize
    cfg.INPUT.RANDOM_FLIP = "none"
    cfg.INPUT.MIN_SIZE_TRAIN = (1024,)
    cfg.INPUT.MAX_SIZE_TRAIN = 1024
    cfg.INPUT.MIN_SIZE_TEST = 1024
    cfg.INPUT.MAX_SIZE_TEST = 1024
    cfg.INPUT.CROP.ENABLED = False
    
    
    cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
    cfg.SOLVER.BASE_LR = 0.005 
    cfg.SOLVER.MAX_ITER = 3000

    return cfg

In [ ]:
# LOAD DATA
register_coco_instances("cghd_train", {}, "../CGHD-COCO/annotations/train.json", "../CGHD-COCO/train")
register_coco_instances("cghd_test", {}, "../CGHD-COCO/annotations/test.json", "../CGHD-COCO/test")
register_coco_instances("cghd_val", {}, "../CGHD-COCO/annotations/val.json", "../CGHD-COCO/val")

In [ ]:
visualize_annotations()

## DETR R50

In [ ]:
cfg = get_default_cfg(class_names)

cfg.OUTPUT_DIR = os.path.join("..", "detectron_models", "detr_R_50")
cfg.MODEL.WEIGHTS = "detr_r50.pth"
cfg.SOLVER.MAX_ITER = 5000
cfg.SOLVER.BASE_LR = 0.001 

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = create_trainer_validation_best_checkpoint(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
visualize_predictions(cfg)

In [ ]:
evaluate_predictions(cfg)

In [ ]:
move_train_artifacts_to_drive(cfg)

## DETR R50 DC5

In [ ]:
cfg = get_default_cfg(class_names)

cfg.OUTPUT_DIR = os.path.join("..", "detectron_models", "detr_R_50_DC5")
cfg.MODEL.WEIGHTS = "detr_r50_dc5.pth"
cfg.SOLVER.MAX_ITER = 5000
cfg.SOLVER.BASE_LR = 0.001

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = Trainer(cfg, ) 
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
visualize_predictions(cfg)

In [ ]:
evaluate_predictions(cfg)

In [ ]:
move_train_artifacts_to_drive(cfg)